In [1]:
!pip3 install httpx bs4 pillow tqdm -q

In [2]:
import httpx
from bs4 import BeautifulSoup
import pathlib
import time
import random
import os
import sys
import asyncio
import typing as tp
import json
from PIL import Image
from io import BytesIO
from tqdm import tqdm

In [35]:
BASE_URL = "https://lib.msk.misis.ru/elib"
DATA_PATH  = pathlib.Path("../data/files/elib-misis")
USERNAME = "2104603"
PASSWORD = "Алексей"
limits = httpx.Limits(max_keepalive_connections=100, max_connections=50)
client = httpx.AsyncClient(timeout=60, limits=limits)

In [4]:
async def login(client: httpx.AsyncClient, username: str, password: str) -> dict[tp.Any, tp.Any]:
    response = await client.post(
        url=BASE_URL + "/login.php",
        data={
            "eye_1": 0,
            "eye_2": 0,
            "eye_3": 0,
            "login": username,
            "password": password,
            "language": "ru",
        },
    )
    if response.status_code != 302:
        raise ValueError
    print(response.status_code, response.content)
    with open('foo.html', "wb") as file:
        file.write(response.content)
    return dict(response.cookies)

In [65]:
async def read_folder(client: httpx.AsyncClient, folder_id: int = 1, folder_path: pathlib.Path = DATA_PATH/"root", depth: int = 0, max_depth: int = 4) -> dict[tp.Any, tp.Any]:
    if depth == max_depth:
        return dict()
    print(folder_path, not os.path.isdir(  folder_path), depth)
    try:
        os.mkdir( folder_path)
    except Exception as e:
        print(e)
        pass

    response = await client.post(
        BASE_URL + "/libs/api.php",
        data={
            "action": "Read_Folders",
            "parent_id": str(folder_id),
        },
    )

    for folder in response.json()["data"]["folders"]:
        new_folder_name  = f"{folder['id']}_{folder['name']}"
        time.sleep(0.3)
        await read_folder(client, folder_id=int(folder["id"]), folder_path=folder_path / new_folder_name, depth=depth + 1, max_depth=max_depth)
    if response.status_code != 200:
        raise ValueError
    return response.json()

# root = await read_folder(client, max_depth=12)

../data/files/elib-misis/root False 0
[Errno 17] File exists: '../data/files/elib-misis/root'
../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС False 1
[Errno 17] File exists: '../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС'
../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС/188_Технические науки False 2
[Errno 17] File exists: '../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС/188_Технические науки'
../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС/188_Технические науки/672_2006 False 3
[Errno 17] File exists: '../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС/188_Технические науки/672_2006'
../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС/188_Технические науки/673_2007 False 3
[Errno 17] File exists: '../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС/188_Технические науки/673_2007'
../data/files/elib-misis/root/187_Авторефераты диссертаций МИСИС/188_Технические н

In [24]:
async def read_in_folder(client: httpx.AsyncClient, folder_id: int, start: int = 0, limit:int = 15) -> dict[tp.Any, tp.Any] | None:
    response = await client.post(
        BASE_URL + "/libs/api.php",
        data={
            "action": "ReadInFolder",
            "folder_id": folder_id,
            "start": start,
            "limit": limit,
            "sort_id": 0,
            "orderby": ["asc"],
        },
    )
    print("read_in_folder", response.status_code)
    if response.json()["error"]:
        return
    return response.json()["data"]

def get_name(metadata: dict) -> str:
    name = "unknown"
    if "id" in metadata.keys() and "value" in metadata.keys() and metadata["id"] == 28:
        return metadata["value"]
    return name

async def get_files_in_folder(client: httpx.AsyncClient, folder: pathlib.Path) -> dict:
    if folder.name == "root":
        folder_id = 1
    else:
        folder_id = int(folder.name.split("_")[0])
    docs_info = dict()
    start, limit = 0, 15

    data = await read_in_folder(client, folder_id, start, limit)
    if data is None or not "count" in data.keys():
        return docs_info
    cnt:int = data["count"]
    print(f"found: {cnt} documents in folder: {folder_id}")
    docs: list[str] = []

    for doc in data["docs"]:
        docs_info[doc["id"]] = doc
        # docs.append(f"{doc['id']}_{get_name(doc)}")
    start += limit

    while start < cnt:
        data = await read_in_folder(client, folder_id, start, limit)
        if data is None:
            break

        for doc in data["docs"]:
            docs_info[doc["id"]] = doc
            # docs.append(f"{doc['id']}_{get_name(doc)}")
        start += limit

    return docs_info

# folder: pathlib.Path = DATA_PATH / "root" / "342_Сборники научных трудов МИСИС"
# docs = await get_files_in_folder(client, folder)
# docs, len(docs)

found: 216 documents in folder: 342


({'6128': {'id': '6128',
   'info': [{'id': '28',
     'name': 'Авторы',
     'value': 'МИС, Самарин, А. М.',
     'document_field_id': '28',
     'title': 'Вып.  1: Новое в технологических процессах металлургического производства'},
    {'id': '29', 'name': 'УДК', 'value': None, 'document_field_id': None},
    {'id': '30',
     'name': 'Год издания',
     'value': '1935',
     'document_field_id': '30'},
    {'id': '31',
     'name': 'Издательство',
     'value': 'ОНТИ',
     'document_field_id': '31'},
    {'id': '32',
     'name': 'Количество страниц',
     'value': '335 с.',
     'document_field_id': '32'},
    {'id': '35', 'name': 'Покупка', 'value': None, 'document_field_id': None}],
   'filetypes': '',
   'stat_show': '83'},
  '6132': {'id': '6132',
   'info': [{'id': '28',
     'name': 'Авторы',
     'value': 'МИС, Самарин, А. М.',
     'document_field_id': '28',
     'title': 'Вып.  2: Электрометаллургия'},
    {'id': '29', 'name': 'УДК', 'value': None, 'document_field_id': No

In [12]:
root_f = DATA_PATH / "root"
root_f.name, folder.name

('root', '69_601-700')

In [30]:
async def get_folder_docs(client: httpx.AsyncClient, root_path: pathlib.Path) -> None:
    json_path = root_path / "info.json"
    if not os.path.exists(json_path):
        docs = await get_files_in_folder(client, root_path)
        print("writing", len(docs), root_path)
        with open(json_path, "w", encoding="utf-8") as folder_info:
            json.dump(docs, folder_info, ensure_ascii=False)

    files_and_folders = os.listdir(root_path)
    for entry in files_and_folders:
        new_path = root_path / entry
        if os.path.isdir(new_path):
            print(new_path)
            await get_folder_docs(client, new_path)


In [36]:
await get_folder_docs(client, DATA_PATH / "root")

../data/files/elib-misis/root/393_УМКД
../data/files/elib-misis/root/393_УМКД/394_Электрооборудование
../data/files/elib-misis/root/393_УМКД/2548_Конструкционные наноматериалы
../data/files/elib-misis/root/393_УМКД/3516_Практическая фонетика английского языка
../data/files/elib-misis/root/393_УМКД/3762_Технологии материалов электроники
../data/files/elib-misis/root/342_Сборники научных трудов МИСИС
../data/files/elib-misis/root/3783_Издания по договору
../data/files/elib-misis/root/655_Сборники научных трудов МИЦМиЗ им. М.И.Калинина
../data/files/elib-misis/root/335_Филиалы МИСИС
../data/files/elib-misis/root/335_Филиалы МИСИС/334_Старооскольский технологический институт им. А.А.Угарова
../data/files/elib-misis/root/335_Филиалы МИСИС/334_Старооскольский технологический институт им. А.А.Угарова/347_Кафедра Высшей математики
../data/files/elib-misis/root/335_Филиалы МИСИС/334_Старооскольский технологический институт им. А.А.Угарова/351_Кафедра Технологии и оборудования в металлургии и ма

In [60]:
async def download_page(client: httpx.AsyncClient, document_id: int = 0, page: int = 0):
    await asyncio.sleep(random.randint(3, 6) / 10)
    response = await client.get(
        "https://lib.msk.misis.ru/elib/libs/view.php",
        params={
            "id":document_id,
            "page":page,
            "type": "small/fast",
        }
    )
    if response.status_code != 200:
        return None
    return response.content


async def get_page_cnt(client: httpx.AsyncClient, document_id: int = 0) -> int:
    response = await client.get(
        "https://lib.msk.misis.ru/elib/view.php",
        params={
            "id":document_id,
            "type":"small/fast",
        }
    )
    if response.status_code != 200:
        return 0
    soup = BeautifulSoup(response.content, "html.parser")
    select_element = soup.find('select', {'id': 'select_page'})
    # Count the number of <option> elements within the <select>
    return  len(select_element.find_all('option'))


def merge_image_bytes_to_pdf(image_bytes_list, output_pdf_path):
    # Convert each byte stream to an Image object and ensure it's in RGB mode
    images = [
        Image.open(BytesIO(img_bytes)).convert("RGB") for img_bytes in image_bytes_list
    ]

    # Save all images as a single PDF
    first_image = images[0]
    first_image.save(output_pdf_path, save_all=True, append_images=images[1:])


async def download_document(client: httpx.AsyncClient, document_id: int = 0, output_path: pathlib.Path = DATA_PATH / "books"):
    page_cnt = await get_page_cnt(client, document_id)
    print(f"found {page_cnt} in document {document_id}")
    if page_cnt == 0:
        return None

    if not os.path.exists(output_path):
        os.mkdir(output_path)

      # Download pages concurrently
    download_tasks = [download_page(client, document_id, page) for page in range(page_cnt)]
    page_contents = await asyncio.gather(*download_tasks)

    merge_image_bytes_to_pdf(
        page_contents,
        output_path / f"{document_id}.pdf"
    )

async def download_lib(client: httpx.AsyncClient, root_path: pathlib.Path = DATA_PATH / "root") -> None:
    json_path = root_path / "info.json"
    if not os.path.exists(json_path):
        return

    with open(json_path, "r", encoding="utf-8") as folder_info:
        data = json.load(folder_info)
    # tasks = []
    for document_id in tqdm(data.keys()):
        if os.path.exists(DATA_PATH / "books" / f"{document_id}.pdf"):
            continue
        # print(f"creating new task for {DATA_PATH / "books" / f"{document_id}.pdf"}")
        # try:
        # tasks.append(download_document(client=client, document_id=document_id))
        # except Exception as e:
            # print(e, document_id)
        await download_document(client=client, document_id=document_id)
    # if tasks:
    #     print(f"starting {len(tasks)}")
    #     await asyncio.gather(*tasks)

    files_and_folders = os.listdir(root_path)
    for entry in files_and_folders:
        if entry == "info.json":
            continue
        new_path = root_path / entry
        if os.path.isdir(new_path):
            await download_lib(client, new_path)

In [61]:
cookies = await login(client, USERNAME, PASSWORD)
cookies

302 b''


{}

In [59]:
while True:
    try:
        await download_lib(client)
        break
    except Exception as e:
        print(e)
        time.sleep(5)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 22/22 [00:00<00:00, 39705.12it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 75/75 [00:00<00:00, 69258.65it/s]
0it [00:00, ?it/s]
100%|██████████| 99/99 [00:01<00:00, 70.86it/s]


found 0 in document 8400


  0%|          | 0/196 [00:00<?, ?it/s]

found 359 in document 3176


  4%|▎         | 7/196 [00:08<04:01,  1.28s/it]

...

found 281 in document 12168


  8%|▊         | 16/196 [01:48<34:15, 11.42s/it]

found 425 in document 12203


  9%|▊         | 17/196 [02:01<35:28, 11.89s/it]

found 434 in document 12077
